In [ ]:
from sklearn.neighbors import NearestNeighbors, kneighbors_graph
from sklearn.manifold import trustworthiness
from scipy.sparse.csgraph import dijkstra
from scipy.spatial import distance_matrix
from sklearn.manifold import MDS
import matplotlib.pyplot as plt
import multiprocessing as mp
from tqdm.auto import tqdm
import tensorflow as tf
import numpy as np
import random
import queue
import json


# Antenna definitions
ASSIGNMENTS = [
	[0, 13, 31, 29, 3, 7, 1, 12 ],
	[30, 26, 21, 25, 24, 8, 22, 15],
	[28, 5, 10, 14, 6, 2, 16, 18],
	[19, 4, 23, 17, 20, 11, 9, 27]
]

ANTENNACOUNT = np.sum([len(antennaArray) for antennaArray in ASSIGNMENTS])

# Optimized version: Calibration constants precomputed once
def load_calibrate_timedomain(path, offset_path):
    with open(offset_path, "r") as offsetfile:
        offsets = json.load(offsetfile)

    # Precompute the calibration tensors
    fft_len = 1024
    freq_range = 2 * np.pi * np.arange(fft_len, dtype=np.float32) / fft_len
    sto_offset = tf.constant(offsets["sto"], dtype=tf.float32)[:, None] * freq_range
    cpo_offset = tf.constant(offsets["cpo"], dtype=tf.float32)[:, None]
    calib_multiplier = tf.exp(tf.complex(0.0, sto_offset + cpo_offset))

    def record_parse_function(proto):
        record = tf.io.parse_single_example(proto, {
            "csi": tf.io.FixedLenFeature([], tf.string),
            "pos-tachy": tf.io.FixedLenFeature([], tf.string),
            "time": tf.io.FixedLenFeature([], tf.float32),
        })

        csi = tf.io.parse_tensor(record["csi"], out_type=tf.float32)
        csi = tf.complex(csi[:, :, 0], csi[:, :, 1])
        csi = tf.signal.fftshift(csi, axes=1)

        position = tf.io.parse_tensor(record["pos-tachy"], out_type=tf.float64)
        time = record["time"]
        return csi, position[:2], time

    def apply_calibration(csi, pos, time):
        csi = tf.multiply(csi, calib_multiplier)
        return csi, pos, time

    def csi_time_domain(csi, pos, time):
        csi = tf.signal.fftshift(tf.signal.ifft(tf.signal.fftshift(csi, axes=1)), axes=1)
        return csi, pos, time

    def cut_out_taps(tap_start, tap_stop):
        def func(csi, pos, time):
            return csi[:, tap_start:tap_stop], pos, time
        return func

    def order_by_antenna(csi, pos, time):
        csi = tf.stack([tf.gather(csi, indices) for indices in ASSIGNMENTS])
        return csi, pos, time

    dataset = tf.data.TFRecordDataset(path)
    dataset = dataset.map(record_parse_function, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(apply_calibration, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(csi_time_domain, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(cut_out_taps(507, 520), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(order_by_antenna, num_parallel_calls=tf.data.AUTOTUNE)

    return dataset

inputpaths = [
	{
		"tfrecords" : "dichasus/dichasus-cf02.tfrecords",
		"offsets" : "dichasus/reftx-offsets-dichasus-cf02.json"
	},
	{
		"tfrecords" : "dichasus/dichasus-cf03.tfrecords",
		"offsets" : "dichasus/reftx-offsets-dichasus-cf03.json"
	},
	{
		"tfrecords" : "dichasus/dichasus-cf04.tfrecords",
		"offsets" : "dichasus/reftx-offsets-dichasus-cf04.json"
	}
]

full_dataset = load_calibrate_timedomain(inputpaths[0]["tfrecords"], inputpaths[0]["offsets"])

for path in inputpaths[1:]:
	full_dataset = full_dataset.concatenate(load_calibrate_timedomain(path["tfrecords"], path["offsets"]))


In [ ]:
import h5py
import numpy as np

# Path to your HDF5 file
file_path = '5G_dataset/5G_training_data.hdf5'

# Create an empty dictionary to store NumPy arrays
data_dict = {}

# Open the HDF5 file
with h5py.File(file_path, 'r') as hdf:
    # Iterate over all datasets in the file
    for key in hdf.keys():
        dataset = hdf[key][:]
        data_dict[key] = dataset  # Store each dataset into the dictionary

# Now, you can access individual datasets as NumPy arrays
A_ID = data_dict['A_ID']
B_ID = data_dict['B_ID']
CIR_I = data_dict['CIR_I']
CIR_R = data_dict['CIR_R']
POS_X = data_dict['POS_X']
POS_Y = data_dict['POS_Y']
TD = data_dict['TD']
TD_OFFSET = data_dict['TD_OFFSET']
TIME_STAMP = data_dict['TIME_STAMP']

# For verification, print shapes of arrays
for key, value in data_dict.items():
    print(f"{key} shape: {value.shape}")


In [ ]:
CIR=CIR_R+ 1j*CIR_I

In [ ]:
CIR

In [ ]:
A_ID

In [ ]:
B_ID

In [ ]:
CIR.shape

In [ ]:
POS_X,POS_Y

In [ ]:
grt_pos=np.stack((POS_X, POS_Y), axis=1)

In [ ]:
grt_pos

In [ ]:
groundtruth_positions = []
csi_time_domain = []
timestamps = []
for csi, pos, time in full_dataset.batch(1000):
	csi_time_domain.append(csi.numpy())
	groundtruth_positions.append(pos.numpy())
	timestamps.append(time.numpy())

csi_time_domain = np.concatenate(csi_time_domain)
groundtruth_positions = np.concatenate(groundtruth_positions)
timestamps = np.concatenate(timestamps)

In [ ]:
csi_time_domain.shape

In [ ]:
groundtruth_positions.shape

In [ ]:
csi_time_domain[0][0][0]

In [ ]:
def feature_extraction_F1(csi_matrix):
    csi_ifft = np.fft.ifft(csi_matrix, axis=1)
    R = csi_ifft * np.conj(csi_ifft)
    r = R.flatten()
    r_real = np.real(r)
    r_imag = np.imag(r)
    r_R = np.concatenate((r_real, r_imag))
    feature_vector = r_R / np.linalg.norm(r_R)
    return feature_vector
m=csi_time_domain.shape[0]
reshaped_csi = csi_time_domain.reshape(m, 32, 13)
features_F1 = np.array([feature_extraction_F1(csi) for csi in reshaped_csi])
print("Shape of Features (F1):", features_F1.shape)

In [ ]:
reshaped_csi[0][0]

In [ ]:
CIR[0]

In [ ]:
def feature_extraction_F1_cir(cir_matrix):
    R = cir_matrix * np.conj(cir_matrix)
    r = R.flatten()
    r_real = np.real(r)
    r_imag = np.imag(r)
    r_R = np.concatenate((r_real, r_imag))
    feature_vector = r_R / np.linalg.norm(r_R)
    return feature_vector

features_F1_cir = np.array([feature_extraction_F1_cir(cir) for cir in CIR])
print("Shape of Features (F1_cir):", features_F1_cir.shape)

In [ ]:
features_F1_cir[0]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, x1, y_train, y1 = train_test_split(features_F1_cir, grt_pos, test_size=0.4, random_state=42)
X_test, X_pred, y_test, y_pred = train_test_split(x1, y1, test_size=0.5, random_state=42)

In [ ]:
# plot the ground truth positions
plt.figure(figsize=(10, 10))
plt.scatter(grt_pos[:, 0], grt_pos[:, 1], c='blue', label='Ground Truth', alpha=0.5)
plt.title('Ground Truth Positions')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
from scipy.special import softmax

def build_grid(grid_size=30, space_bounds=None):
    (xmin, xmax), (ymin, ymax) = space_bounds
    x_lin = np.linspace(xmin, xmax, grid_size)
    y_lin = np.linspace(ymin, ymax, grid_size)
        
    xv, yv = np.meshgrid(x_lin, y_lin)
    grid_points = np.stack([xv.ravel(), yv.ravel()], axis=1)
    return grid_points

def compute_soft_probability_maps(positions, grid_points, temperature=.1):
    N = positions.shape[0]
    K = grid_points.shape[0]
    dists = np.linalg.norm(positions[:, np.newaxis, :] - grid_points[np.newaxis, :, :], axis=2)
    sim_scores = -dists / temperature
    prob_maps = softmax(sim_scores, axis=1)
    return prob_maps

def estimate_positions_from_maps(prob_maps, grid_points):
    return np.matmul(prob_maps, grid_points)

def generate_all_probability_maps(positions, grid_size=30, temperature=.1):
    x_min, x_max = positions[:, 0].min(), positions[:, 0].max()
    y_min, y_max = positions[:, 1].min(), positions[:, 1].max()
    print("Position bounds:", (x_min, x_max), (y_min, y_max))
    grid_points = build_grid(grid_size, space_bounds=((x_min, x_max), (y_min, y_max)))
    print("Grid points shape:", grid_points.shape)
    prob_maps = compute_soft_probability_maps(positions, grid_points, temperature=temperature)

    est_locations = estimate_positions_from_maps(prob_maps, grid_points)

    return prob_maps, est_locations, grid_points

In [ ]:
def plot_heatmap(prob_map, grid_points, title="Heatmap"):
    plt.figure(figsize=(8, 6))
    plt.imshow(prob_map.reshape(30, 30), extent=(grid_points[:, 0].min(), grid_points[:, 0].max(),
                                                  grid_points[:, 1].min(), grid_points[:, 1].max()),
               origin='lower', cmap='hot', alpha=0.7)
    plt.colorbar(label='Probability')
    plt.title(title)
    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.show()

In [ ]:
prob_map_test, est_locations_test, grid_points_test = generate_all_probability_maps(y_test, grid_size=100, temperature=0.1)
prob_map_train, est_locations_train, grid_points_train = generate_all_probability_maps(y_train, grid_size=100, temperature=0.1)
prob_map_pred, est_locations_pred, grid_points_pred = generate_all_probability_maps(y_pred, grid_size=100, temperature=0.1)

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(y_test[:, 0], y_test[:, 1], c='blue', label='Ground Truth (Test)', alpha=0.5)
plt.scatter(est_locations_test[:, 0], est_locations_test[:, 1], c='red', label='Estimated (Test)', alpha=0.5)
plt.title('Estimated vs Ground Truth Positions (Test Set)')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from keras import layers
import tensorflow as tf

class NeuralNetwork(tf.keras.Model):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = layers.Dense(2048, input_dim=input_size)
        self.bn1 = layers.BatchNormalization()
        self.fc2 = layers.Dense(1024)
        self.bn2 = layers.BatchNormalization()
        # self.fc3 = layers.Dense(512)
        # self.fc4 = layers.Dense(512)
        # self.fc5 = layers.Dense(512)
        self.fc3 = layers.Dense(output_size, activation='softmax')

    def call(self, inputs):
        x = self.fc1(inputs)
        x = self.bn1(x)
        x = tf.nn.relu(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = tf.nn.relu(x)

        x = self.fc3(x)
        return x

# Set output_size to the number of grid points (e.g., 900 for 30x30 grid)
output_size = grid_points_train.shape[0]  # grid_points from your earlier code
print(output_size)

In [ ]:
X_train.shape

In [ ]:
X_train.mean(axis=1)

In [ ]:
prob_map_train.shape

In [ ]:
# predicted_maps = model.predict(X_pred)
# plot_heatmap(predicted_maps[2], grid_points_pred, title="Predicted Heatmap for First Test Position")
# print(f"Estimated :- {estimate_positions_from_maps(predicted_maps, grid_points_pred)[2]}")
# print(f"Ground Truth :- {y_pred[2]}")
# plot_heatmap(prob_map_pred[2], grid_points_pred, title="Heatmap for First Test Position")
# predicted_positions = np.matmul(predicted_maps, grid_points_pred)
# mae = np.mean(np.abs(y_pred - predicted_positions), axis=0)
# print("Mean Absolute Error (Position Predictions):", mae)

In [ ]:
# import tensorflow as tf
# from keras.layers import Input, Dense, BatchNormalization, Dropout
# from keras.models import Model

# def build_denoising_autoencoder(input_dim):
#     """
#     Builds a denoising autoencoder model.

#     Args:
#         input_dim: Dimension of the input features (e.g., 832 for features_F1).

#     Returns:
#         autoencoder: The denoising autoencoder model.
#     """
#     # Input layer
#     input_data = Input(shape=(input_dim,), name="input")

#     # Encoder
#     x = Dense(512, activation="relu", name="encoder_dense_1")(input_data)
#     x = BatchNormalization(name="encoder_bn_1")(x)
#     x = Dropout(0.2, name="encoder_dropout_1")(x)

#     x = Dense(256, activation="relu", name="encoder_dense_2")(x)
#     x = BatchNormalization(name="encoder_bn_2")(x)
#     x = Dropout(0.2, name="encoder_dropout_2")(x)

#     x = Dense(128, activation="relu", name="encoder_dense_3")(x)
#     x = BatchNormalization(name="encoder_bn_3")(x)
#     x = Dropout(0.2, name="encoder_dropout_3")(x)

#     x = Dense(64, activation="relu", name="encoder_dense_4")(x)
#     x = BatchNormalization(name="encoder_bn_4")(x)
#     x = Dropout(0.2, name="encoder_dropout_4")(x)

#     encoded = Dense(32, activation="relu", name="encoder_latent")(x)  # Latent space

#     # Decoder
#     x = Dense(64, activation="relu", name="decoder_dense_1")(encoded)
#     x = BatchNormalization(name="decoder_bn_1")(x)
#     x = Dropout(0.2, name="decoder_dropout_1")(x)

#     x = Dense(128, activation="relu", name="decoder_dense_2")(x)
#     x = BatchNormalization(name="decoder_bn_2")(x)
#     x = Dropout(0.2, name="decoder_dropout_2")(x)

#     x = Dense(256, activation="relu", name="decoder_dense_3")(x)
#     x = BatchNormalization(name="decoder_bn_3")(x)
#     x = Dropout(0.2, name="decoder_dropout_3")(x)

#     x = Dense(512, activation="relu", name="decoder_dense_4")(x)
#     x = BatchNormalization(name="decoder_bn_4")(x)
#     x = Dropout(0.2, name="decoder_dropout_4")(x)

#     decoded = Dense(input_dim, activation="linear", name="decoder_output")(x)  # Reconstruct clean features

#     # Autoencoder model
#     autoencoder = Model(input_data, decoded, name="DenoisingAutoencoder")

#     return autoencoder

# # Build the denoising autoencoder
# input_dim = 832  # Dimension of the input features
# autoencoder = build_denoising_autoencoder(input_dim)

# # Compile the autoencoder
# autoencoder.compile(optimizer="adam", loss="mse")

# # Print the autoencoder summary
# autoencoder.summary()

In [ ]:
grid_points = build_grid(grid_size=30, space_bounds=((groundtruth_positions[:, 0].min(), groundtruth_positions[:, 0].max()),
                                                    (groundtruth_positions[:, 1].min(), groundtruth_positions[:, 1].max())))
# Plot the grid points
plt.figure(figsize=(10, 10))
plt.scatter(grid_points[:, 0], grid_points[:, 1], c='red', label='Grid Points', alpha=0.5)
plt.scatter(groundtruth_positions[:, 0], groundtruth_positions[:, 1], c='blue', label='Ground Truth', alpha=0.5)
plt.title('Grid Points and Ground Truth Positions')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
csi_time_domain_original = csi_time_domain
groundtruth_positions_original = groundtruth_positions
features_F1_original=features_F1

In [ ]:
prob_map_train_original = prob_map_train

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve
from keras.callbacks import EarlyStopping
num_runs = 1  # or however many runs you want
num_lp = 5     # since lp_values = [1, 2, 3, 4]

mae_noise_runs = np.zeros((num_runs, num_lp))

for run in range(num_runs):
    mae_noise_list = []
    mae_denoise_list = []
    lp_values = []
    temp=20000


    csi_time_domain = csi_time_domain_original
    groundtruth_positions = groundtruth_positions_original

    prob_map_train = prob_map_train_original

    prob_map_train = prob_map_train[:temp,:]

    csi_time_domain=csi_time_domain[:temp,:,:]
    groundtruth_positions= groundtruth_positions[:temp,:]

    for x in range(1, 50,10):
        lp_values.append(x)

        # Parameters
        W = 1024
        modulation_order = 16
        C = W // 8
        L = 13
        Lp = x
        noise_power = 0.1
        B = 32

        # Generate frequency-domain symbols
        data = np.random.randint(0, modulation_order, W)
        qam_symbols = (2 * (data % 4) - 3) + 1j * (2 * (data // 4) - 3)
        qam_symbols /= np.sqrt(10)

        time_domain_signal = np.fft.ifft(qam_symbols)

        # Zero-mean Gaussian for amplitude
        A_k = np.random.normal(loc=0.0, scale=1.0, size=Lp)
        # Zero-mean Gaussian for phase
        phi_k = np.random.normal(loc=0.0, scale=1.0, size=Lp)

        p_bar = A_k * np.exp(1j * phi_k)
        # p_bar /= np.linalg.norm(p_bar)

        perturbed_signal = np.convolve(p_bar, time_domain_signal, mode="full")[:W]
        cyclic_prefix = perturbed_signal[-C:]
        transmit_signal = np.concatenate([cyclic_prefix, perturbed_signal])

        csi_time_domain = csi_time_domain.reshape(temp, 32, 13)
        num_csi_instances = csi_time_domain.shape[0]

        y_real_all = np.zeros((num_csi_instances, B, W + C), dtype=complex)
        H_pert_all = np.zeros((num_csi_instances, B, W), dtype=complex)

        for i in range(num_csi_instances):
            H_real = csi_time_domain[i]
            for b in range(B):
                y_real = convolve(H_real[b], transmit_signal, mode='full')[:W + C]
                noise = np.sqrt(noise_power / 2) * (np.random.randn(W + C) + 1j * np.random.randn(W + C))
                y_real_noisy = y_real + noise
                y_real_no_cp = y_real_noisy[C:]

                y_freq = np.fft.fft(y_real_no_cp)
                s_freq = np.fft.fft(time_domain_signal)

                H_pert_all[i, b, :] = y_freq / s_freq
                H_pert_time = np.fft.ifft(H_pert_all[i, b, :])[:L]
                csi_time_domain[i, b, :] = H_pert_time

        csi_time_domain = csi_time_domain.reshape(temp, 4, 8, 13)
        reshaped_csi = csi_time_domain.reshape(temp, 32, 13)
        features_F1 = np.array([feature_extraction_F1(csi) for csi in reshaped_csi])
        X_train = features_F1
        y_train = features_F1_original[:temp,:]

        # history = autoencoder.fit(
        #     X_train, y_train,
        #     epochs=30,
        #     batch_size=32,
        #     validation_split=0.2,
        #     verbose=0
        # )
        print(features_F1.shape)
        model = NeuralNetwork(input_size=832, output_size=output_size)

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
        
        # early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=0, restore_best_weights=True)
        print(X_train.shape, prob_map_train.shape)
        history = model.fit(X_train, prob_map_train, epochs=50, batch_size=128, validation_split=0.2)

        x_noise = model.predict(features_F1)
        # features_denoise = autoencoder.predict(features_F1)
        # x_denoise = model.predict(features_denoise)
        # x_denoise = np.matmul(x_denoise, grid_points)
        x_noise = np.matmul(x_noise, grid_points)
        mae_noise = np.mean(np.sqrt((groundtruth_positions[:, 0] - x_noise[:, 0])**2 + (groundtruth_positions[:, 1] - x_noise[:, 1])**2))
        # mae_denosie = np.mean(np.sqrt((groundtruth_positions[:, 0] - x_denoise[:, 0])**2 + (groundtruth_positions[:, 1] - x_denoise[:, 1])**2))

        mae_noise_list.append(mae_noise)
        # mae_denoise_list.append(mae_denosie)

# Plot MAE vs Perturbation Length
mae_noise_runs= np.mean(mae_noise_runs, axis=0)
plt.plot(lp_values, mae_noise_list, label='MAE (Noisy Features)', marker='o')
# plt.plot(lp_values, mae_denoise_list, label='MAE (Denoised Features)', marker='s')
plt.xlabel('Perturbation Length (Lp)')
plt.ylabel('Mean Absolute Error (MAE)')
plt.title('MAE vs Perturbation Length')
plt.legend()
plt.grid(True)
plt.show()